In [2]:
import pickle
import numpy as np
import pandas as pd

import gensim
from gensim.models import Phrases
from gensim.models.phrases import Phraser

from collections import defaultdict

import sys
sys.path.append('../src/clio/')
from utils import *
from text_preprocessing import *
from clio import *

In [4]:
# Read data
df1 = pd.read_json('../data/raw/corex_matched_noOAG.json')
df1 = df1[['institutes', 'arxiv_categories', 'arxiv_raw_summary', 'arxiv_id', 'date']]
df1['Source ID'] = 'arxiv'
df1.rename(index=str, inplace=True, columns={'arxiv_id':'Row ID', 'date':'Start Date', 'institutes':'Name', 'arxiv_raw_summary':'Description', 'arxiv_id':'Row ID'})

df2 = pd.read_csv('../data/raw/gh_cb_gdb_v1.csv', compression='gzip')

# concat df1, df2
df = pd.concat([df2, df1])
df = df[df.Description.isnull()==False]
df.reset_index(inplace=True, drop=True)

print('Data shape: {}'.format(df.shape))

/Users/konstantinosstathoulopoulos/anaconda3/envs/py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (2,5,6,8,9,10,11,12,13,14,15,16,19,20,23) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/konstantinosstathoulopoulos/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:10: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



Data shape: (7139532, 25)


In [8]:
df.to_csv('../data/processed/df.csv')

In [8]:
nlproc = TextPreprocessing()

In [9]:
def rwjf_sent(text):
    """Remove first sentence of RWJF grants."""
    txt = []
    for sent in text.split('.'):
        if ':' in sent:
            txt.extend(sent.split(':'))
        else:
            txt.append(sent)
    if len(txt) > 1:
        return '. '.join(txt[1:])
    else:
        return ' '.join(txt)

In [10]:
df['Description'] = [desc if df.loc[i, 'GDB Dataset ID'] != 'rwjf' else rwjf_sent(desc) for i, desc in enumerate(df['Description']) ]

In [8]:
%%time
documents = [flatten_lists(nlproc.tokenize_document(doc)) for doc in list(df['Description'])]

CPU times: user 35min 4s, sys: 17.6 s, total: 35min 21s
Wall time: 35min 26s


In [9]:
%%time
# bigrams
phrases = gensim.models.Phrases(documents, min_count=2, delimiter=b'_')
bigram = gensim.models.phrases.Phraser(phrases)
docs_bi = list(bigram[documents])

# trigrams
phrases = gensim.models.Phrases(docs_bi, min_count=2, delimiter=b'_')
trigram = gensim.models.phrases.Phraser(phrases)
docs = list(trigram[docs_bi])

CPU times: user 30min 32s, sys: 7min 46s, total: 38min 19s
Wall time: 41min 30s


In [11]:
len(docs)

7139561

In [13]:
import pickle
import os.path

file_path = "../data/processed/docs.pkl"
n_bytes = 2**31
max_bytes = 2**31 - 1
data = bytearray(n_bytes)

## write
bytes_out = pickle.dumps(docs)
with open(file_path, 'wb') as f_out:
    for idx in range(0, len(bytes_out), max_bytes):
        f_out.write(bytes_out[idx:idx+max_bytes])

In [ ]:
%%time
w2v = gensim.models.Word2Vec(docs, size=300, window=10, min_count=2, iter=20)

In [ ]:
w2v.save('../models/w2v.model')